In [ ]:
import os
os.getcwd()
os.chdir("..")

In [ ]:
os.getcwd()


In [ ]:
# Optimizing the chucking process
import os
import pdfplumber
import re


# python PDFProcessing/PDFProccesing.py > outputs/refoutput.txt

text_list = []

with pdfplumber.open("PDF_docs/doc_0.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            text_list.append(text)
            text = text.join(text_list)
text





In [ ]:
parts = re.split(r"\breferences\b", text, flags=re.IGNORECASE, maxsplit=1)

body = parts[0].strip()  # Everything before "References"
reference = parts[1].strip() if len(parts) > 1 else ""  # Everything after "References"



In [ ]:
reference

In [ ]:
# Get meta data

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_2.pdf")
pdf.metadata

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_0.pdf")
metadata = pdf.metadata  # Extrahiere Metadaten
if metadata and "Title" in metadata and metadata["Title"]:
    print( f"Titel: {metadata['Title']}")
else: 
    print(False)

In [ ]:
with pdfplumber.open("PDF_docs/doc_1.pdf") as pdf:
    metadata = pdf.metadata
    print(metadata)

In [1]:
from ollama import ChatResponse, chat
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_chroma import Chroma
import chromadb

In [2]:
import os
os.chdir("..")

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
import chromadb
from langchain_chroma import Chroma

# # 📌 Schritt 1: PDFs laden
# pdf_dir = "PDF_docs/"
# pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]  # Nur PDFs

# all_docs = []
# for pdf_file in pdf_files:
#     pdf_path = os.path.join(pdf_dir, pdf_file)
#     loader = PyPDFLoader(pdf_path)
#     docs = loader.load()
#     all_docs.extend(docs)  # Dokumente speichern

# print(f"✅ {len(all_docs)} Dokumente geladen.")

# # 📌 Schritt 2: Embeddings erstellen
# embeddings = OllamaEmbeddings(model="llama3.2:latest")  # Korrekte Embeddings

# # 📌 Schritt 3: ChromaDB persistent speichern
# persistent_client = chromadb.PersistentClient(path="./chroma_langchain_db")  # Verzeichnis für Speicherung
# collection = persistent_client.get_or_create_collection("collection_name")

# 📌 Schritt 4: Dokumente zu ChromaDB hinzufügen
# doc_texts = [doc.page_content for doc in all_docs]  # Extrahiere den Text
# doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs

# collection.add(ids=doc_ids, documents=doc_texts)  # Speichern in ChromaDB

# # 📌 Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
# vector_store_from_client = Chroma(
#     persist_directory="./chroma_langchain_db",
#     collection_name="collection_name",
#     embedding_function=embeddings,
# )

# print("✅ ChromaDB erfolgreich mit PDFs initialisiert!")


In [3]:
file_path = "PDF_docs/"

def save_doc(file_path):
    # Wir laden die pdf dokumente aus dem direcotire nur die mit .pdf enden
    pdf_files = [f for f in os.listdir(file_path) if f.endswith(".pdf")]  # Nur PDFs

    all_docs = [] # initalisieren eine leere liste
    for pdf_file in pdf_files:
        pdf_path = os.path.join(file_path, pdf_file)
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        all_docs.extend(docs)  # Dokumente speichern
        # Speichern all dokumente in die liste all_docs
    return all_docs
all_docs = save_doc(file_path=file_path)
len(all_docs)

49

In [4]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(all_docs)
texts

[Document(metadata={'producer': 'Adobe PDF Library 9.0; modified using iText 4.2.0 by 1T3XT', 'creator': 'Adobe InDesign CS4 (6.0.6)', 'creationdate': '2012-05-30T17:31:53-04:00', 'moddate': '2025-02-11T09:45:26-08:00', 'trapped': '/False', 'subject': 'N Engl J Med 2012.366:2207-2214', 'title': 'Two Hundred Years of Cancer Research', 'source': 'PDF_docs/doc_0.pdf', 'total_pages': 8, 'page': 0, 'page_label': '2207'}, page_content='T h e n e w  e ng l a n d  j o u r na l  o f m e dic i n e\nn engl j med 366;23 nejm.org june 7, 2012 2207\n                   anniversary article\nTwo Hundred Years of Cancer Research\nVincent T. DeVita, Jr., M.D., and Steven A. Rosenberg, M.D., Ph.D.\nFrom the Yale Comprehensive Cancer \nCenter and Smilow Cancer Hospital at \nYale–New Haven, Yale University School \nof Medicine, and Yale University School \nof Public Health — all in New Haven, CT \n(V.T.D.); the National Cancer Institute, \nNational Institutes of Health, and the \nUniformed Services Universi

In [5]:
def embedding():
    embeddings = OllamaEmbeddings(model="granite-embedding:30m")  # Korrekte Embeddings
    return embeddings

embeddings = embedding() 
embeddings

OllamaEmbeddings(model='granite-embedding:30m', base_url=None, client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [6]:
def persistent_clientChroma():
    persistent_client = chromadb.PersistentClient(path="./chroma_langchain_db")  # Verzeichnis für Speicherung
    collection = persistent_client.get_or_create_collection("collection_name")
    return collection

collection = persistent_clientChroma()
collection

Collection(name=collection_name)

In [7]:
def add_doc_to_Chroma():
    doc_texts = [doc.page_content for doc in texts]  # Extrahiere den Text
    doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs

    collection_db = collection.add(ids=doc_ids, documents=doc_texts)  # Speichern in ChromaDB
    return collection_db
collection_db = add_doc_to_Chroma()
collection_db

In [8]:
def vector_store_from_clientChroma():
    # Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
    vector_store_from_client = Chroma(
        persist_directory="./chroma_langchain_db",
        collection_name="collection_name",
        embedding_function=embeddings,
    )
    return vector_store_from_client

In [9]:
vector_store_from_client= vector_store_from_clientChroma()
vector_store_from_client

In [10]:
results = vector_store_from_client.as_retriever(search_type='similarity',
    k=2
)
print(results)
question = ' to a Gaussian process'
results.invoke(question)

tags=['Chroma', 'OllamaEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x128bc2cf0> search_kwargs={}


[Document(id='doc_23', metadata={}, page_content='Parhi, Bohra, El Biari, Pourya, and Unser\nwhere (52) holds by the mutual independence of the (wk,bk) and (53) holds since the random\nvariables\n(w′\nk,b′\nk) |(w′\nk,b′\nk) ∈supp ψ (55)\nare uniformly distributed on supp ψ. Next, deﬁne the auxiliary functional\nM(ψ) =\n∫\nR\n∫\nsupp ψ\neivψ(w,b) d(w,b) dPV(v). (56)\nWe have that\nˆQwPoi(ψ) = E\n\uf8ee\n\uf8f0\nNψ∏\nk=1\nM(ψ)\n|supp ψ|\n\uf8f9\n\uf8fb\n= E\n[( M(ψ)\n|supp ψ|\n)Nψ\n]\n=\n∞∑\nn=0\n( M(ψ)\n|supp ψ|\n)n(λ|supp ψ|)n\nn! e−λ|supp ψ| (57)\n= e−λ|supp ψ|\n∞∑\nn=0\n(λM(ψ))n\nn!\n= e−λ|supp ψ|eλM(ψ) (58)\n= exp(λ(M(ψ) −|supp ψ|) (59)\n= exp\n(\nλ\n∫\nR\n∫\nSd−1×R\n(\neivψ(z) −1\n)\ndzdPV(v)\n)\n, (60)\nwhere (57) holds since Nψ is a Poisson random variable with mean λ|supp ψ|, (58) holds by\nthe Taylor series expansion of t↦→et, (59) holds since |supp ψ|=\n∫\nsupp ψ1 dz, and (60) holds\nsince z↦→eivψ(z) −1 vanishes outside supp ψ. At this point, we remark that, since PV is a\nL´

In [ ]:
print(len(embeddings.embed_query("test")))  # Gibt die Anzahl der Dimensionswerte aus


In [ ]:
print(all_docs[30].metadata)  # Prüfen, welche Metadaten die Dokumente haben


In [ ]:
print(collection.count())  # Sollte die Anzahl der gespeicherten Dokumente anzeigen


In [ ]:
print(embeddings.embed_query("test"))
